In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle
import random

from collections import defaultdict
from torchvision.models import resnet18
from torchvision.models.resnet import ResNet18_Weights
from torch.optim.lr_scheduler import CosineAnnealingLR
from PIL import Image
from sklearn.model_selection import train_test_split

In [ ]:
torch.manual_seed(2004)
random.seed(2004)
np.random.seed(2004)

In [ ]:
BATCH_SIZE = 16 # Assumed
LR = 0.025 # Same as paper
MOMENTUM = 0.9 # Same as paper
USE_NESTEROV = True # Same as paper
EPOCHS = 100
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
PRINT_INTERVAL = 1000

MODEL_SAVE_DIR = "models"
CLASSES = ("plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

In [ ]:
# Load active learning embeddings and build dataset
embeddings_dir = "embeddings/100_iterations"
if not os.path.exists(f"embeddings/100_iterations"):
    raise ValueError("Please run build_dataset.py first to generate embeddings for active learning")

active_learning_embeddings = {}
for i in range(100):
    with open(f"{embeddings_dir}/embedding_{i}.pkl", "rb") as f:
        embeddings = pickle.load(f)
        active_learning_embeddings[i] = embeddings

print("Embeddings shape:", len(active_learning_embeddings))

all_images = []
all_labels = []
for key, value_dict in active_learning_embeddings.items():
    image = value_dict["image"]
    print(image.flatten().min(), image.flatten().max())
    image = Image.fromarray(image.astype("uint8")) # Convert to PIL image
    label = np.array(value_dict["label"]) # Convert scalar to 1D array
    print(np.array(image).shape, label.shape)
    all_images.append(image)
    all_labels.append(label)

In [ ]:
# Separate into train and validation sets:
train_images, val_images, train_labels, val_labels = train_test_split(all_images, all_labels, test_size=0.2, random_state=2004)

print(f"Number of training images: {len(train_images)}")
print(f"Number of validation images: {len(val_images)}")

In [ ]:
def count_instances_per_class(labels):
    class_counts = defaultdict(int)
    for label in labels:
        class_counts[CLASSES[label.item()]] += 1
    return dict(class_counts)

def print_class_counts(class_counts, labels):
    for c_class, count in class_counts.items():
        print(f"Class: {c_class} | Count: {count} | Percentage: {count / len(labels) * 100:.5f}%")

In [ ]:
train_class_counts = count_instances_per_class(train_labels)
val_class_counts = count_instances_per_class(val_labels)

print("Training class counts:")
print_class_counts(train_class_counts, train_labels)
print("\n")

print("Validation class counts:")
print_class_counts(val_class_counts, val_labels)

In [ ]:
transform = transforms.Compose(
    [
        transforms.RandomCrop(32, padding=0), # Padding not mentioned in paper
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels, transform=None):
        super().__init__()
        self.images = images
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)
        
        label = torch.tensor(label)
        return image, label
    
# train_set = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
train_set = CustomDataset(images=train_images, labels=train_labels, transform=transform)
train_dl = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

val_set = CustomDataset(images=val_images, labels=val_labels, transform=transform)
val_dl = torch.utils.data.DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

test_set = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
test_dl = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [ ]:
print(len(train_dl))
print(len(test_dl))

In [ ]:
data = train_dl.__iter__().__next__()
images, labels = data
print(images.shape, labels.shape)

In [ ]:
def imshow(img):
    img = img / 2 + 0.5 # Unnormalize
    np_img = img.numpy()
    plt.imshow(np.transpose(np_img, (1, 2, 0)))
    plt.show()

data_iter = iter(train_dl)
images, labels = next(data_iter)

imshow(torchvision.utils.make_grid(images))
print(" ".join(f"{CLASSES[labels[j]]:5s}" for j in range(BATCH_SIZE)))

In [ ]:
def create_resnet18_model():
    model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
    model.fc = nn.Linear(512, 10) # Adapt final layer to CIFAR-10 classes
    return model

In [ ]:
model = create_resnet18_model()
model = model.to(DEVICE)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimiser = optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, nesterov=USE_NESTEROV)
scheduler = CosineAnnealingLR(optimiser, T_max=EPOCHS) # T_max is the number of epochs

In [ ]:
print(len(train_dl), len(test_dl))

In [ ]:
num_train_batches = len(train_dl)

In [ ]:
def calculate_metrics(true_positive, false_positive, false_negative, total):
    precision = {}
    recall = {}
    f1_score = {}
    accuracy = {}
        
    for class_name in CLASSES:
        tp = true_positive[class_name]
        fp = false_positive[class_name]
        fn = false_negative[class_name]
        total_count = total[class_name]

        # Accuracy (same as existing code)
        accuracy[class_name] = 100 * float(tp) / total_count if total_count != 0 else 0

        # Precision = TP / (TP + FP)
        precision[class_name] = tp / (tp + fp) if (tp + fp) != 0 else 0

        # Recall = TP / (TP + FN)
        recall[class_name] = tp / (tp + fn) if (tp + fn) != 0 else 0

        # F1 Score = 2 * (Precision * Recall) / (Precision + Recall)
        if (precision[class_name] + recall[class_name]) > 0:
            f1_score[class_name] = 2 * (precision[class_name] * recall[class_name]) / (precision[class_name] + recall[class_name])
        else:
            f1_score[class_name] = 0

    return precision, recall, f1_score, accuracy

In [ ]:
def epoch_forward_pass(model, criterion, optimiser, data_loader, epoch, num_batches, mode="train"):

    total = {class_name: 0 for class_name in CLASSES}
    true_positive = defaultdict(int)
    false_positive = defaultdict(int)
    false_negative = defaultdict(int)

    running_loss = 0.0
    total_examples = 0

    for i, data in enumerate(data_loader):
        inputs, labels = data
        optimiser.zero_grad()

        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        output = model(inputs)

        # Count true positives, false positives, and false negatives
        _, predicted = torch.max(output, 1)
        
        for label, prediction in zip(labels, predicted):
            total[CLASSES[label]] += 1 # Track total for accuracy

            if label == prediction:
                true_positive[CLASSES[label]] += 1
            else:
                false_positive[CLASSES[prediction]] += 1 # Predicted class is wrong
                false_negative[CLASSES[label]] += 1 # Actual class is wrong
        
        loss = criterion(output, labels)
        
        if mode == "train":
            loss.backward()
            optimiser.step()

        running_loss += loss.item()
        total_examples += len(labels)

        if (i % PRINT_INTERVAL == (PRINT_INTERVAL - 1)) or (i == num_batches - 1):
            current_lr = optimiser.param_groups[0]["lr"]
            print(f"Epoch: {epoch + 1} |  Batch: {i + 1}/{num_batches} | Loss: {running_loss / total_examples} | LR: {current_lr}")

    # Calculate metrics after each epoch
    precision, recall, f1_score, accuracy = calculate_metrics(
                                                            true_positive=true_positive, 
                                                            false_positive=false_positive, 
                                                            false_negative=false_negative, 
                                                            total=total
                                                            )
    average_epoch_loss = running_loss / total_examples
    return average_epoch_loss, precision, recall, f1_score, accuracy
    


In [ ]:
for epoch in range(EPOCHS):
    model.train()
    train_loss, train_precision, train_recall, train_f1_score, train_accuracy = epoch_forward_pass(
                                                                                    model=model,
                                                                                    criterion=criterion,
                                                                                    optimiser=optimiser,
                                                                                    data_loader=train_dl,
                                                                                    epoch=epoch,
                                                                                    num_batches=len(train_dl),
                                                                                    mode="train"
                                                                                    )
    # Print metrics for each class:
    print(f"Epoch: {epoch + 1}")
    print(f"Training metrics")
    print(f"Average train loss: {train_loss:.4f}")
    for class_name in CLASSES:
        print(f"Class: {class_name} | Precision: {train_precision[class_name]:.2f} | Recall: {train_recall[class_name]:.2f} | F1 Score: {train_f1_score[class_name]:.2f} | Accuracy: {train_accuracy[class_name]:.2f}")
    
    # Validate model after each epoch
    model.eval()
    with torch.no_grad():
        val_loss, val_precision, val_recall, val_f1_score, val_accuracy = epoch_forward_pass(
                                                                                            model=model,
                                                                                            criterion=criterion,
                                                                                            optimiser=optimiser,
                                                                                            data_loader=val_dl,
                                                                                            epoch=epoch,
                                                                                            num_batches=len(val_dl),
                                                                                            mode="val"
                                                                                            )
    # Print metrics for each class:
    print(f"Validation metrics")
    print(f"Average val loss: {val_loss:.4f}")
    for class_name in CLASSES:
        print(f"Class: {class_name} | Precision: {val_precision[class_name]:.2f} | Recall: {val_recall[class_name]:.2f} | F1 Score: {val_f1_score[class_name]:.2f} | Accuracy: {val_accuracy[class_name]:.2f}")
    print("\n")

    # Update scheduler after each epoch
    scheduler.step()

In [ ]:
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
model_path = f"{MODEL_SAVE_DIR}/fully_supervised_model.pth"
torch.save(model.state_dict(), model_path)

In [ ]:
data_iter = iter(test_dl)
images, labels = next(data_iter)
imshow(torchvision.utils.make_grid(images))
print(f"Ground truth: {', '.join(f'{CLASSES[labels[j]]:5s}' for j in range(BATCH_SIZE))}")

In [ ]:
saved_model = create_resnet18_model()
saved_model.load_state_dict(torch.load(model_path))
saved_model.to(DEVICE)

In [ ]:
output = saved_model(images.to(DEVICE))
_, predicted = torch.max(output, 1)

In [ ]:
print(f"Predictions: {', '.join(f'{CLASSES[predicted[j]]:5s}' for j in range(BATCH_SIZE))}")

In [ ]:
total = {class_name: 0 for class_name in CLASSES}
true_positive = defaultdict(int)
false_positive = defaultdict(int)
false_negative = defaultdict(int)

saved_model.eval()

with torch.no_grad():
    for data in test_dl:
        images, labels = data

        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        output = saved_model(images)

        _, predicted = torch.max(output, 1)
        
        for label, prediction in zip(labels, predicted):
            total[CLASSES[label]] += 1 # Track total for accuracy

            if label == prediction:
                true_positive[CLASSES[label]] += 1
            else:
                false_positive[CLASSES[prediction]] += 1 # Predicted class is wrong
                false_negative[CLASSES[label]] += 1 # Actual class is wrong

In [ ]:
precision, recall, f1_score, accuracy = calculate_metrics(
                                                        true_positive=true_positive, 
                                                        false_positive=false_positive, 
                                                        false_negative=false_negative, 
                                                        total=total
                                                        )

In [ ]:
for class_name in total.keys():
    print(f"Class name: {class_name}")
    print(f"Accuracy: {accuracy[class_name]:.4f}")
    print(f"Precision: {precision[class_name]:.4f}")
    print(f"Recall: {recall[class_name]:.4f}")
    print(f"F1 Score: {f1_score[class_name]:.4f}")
    print("\n")